In [15]:
%pip install pandas


In [4]:
import pandas as pd

In [18]:
pd.set_option("display.width", 1000)  #Prevent column wrapping
df = pd.read_csv("retail_sales_dataset.csv")
print(df.head(20))

    Transaction ID        Date Customer ID  Gender  Age Product Category  Quantity  Price per Unit  Total Amount
0                1  2023-11-24     CUST001    Male   34           Beauty         3              50           150
1                2  2023-02-27     CUST002  Female   26         Clothing         2             500          1000
2                3  2023-01-13     CUST003    Male   50      Electronics         1              30            30
3                4  2023-05-21     CUST004    Male   37         Clothing         1             500           500
4                5  2023-05-06     CUST005    Male   30           Beauty         2              50           100
5                6  2023-04-25     CUST006  Female   45           Beauty         1              30            30
6                7  2023-03-13     CUST007    Male   46         Clothing         2              25            50
7                8  2023-02-22     CUST008    Male   30      Electronics         4              

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


Transform the column names to lower and remove spacing

In [6]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
print(df.columns)

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age', 'product_category', 'quantity', 'price_per_unit', 'total_amount'], dtype='object')


In [7]:
df['date'] = pd.to_datetime(df['date'])
df['customer_id'] = df['customer_id'].astype('string')
df['gender'] = df['gender'].str.capitalize()
df['product_category']
df = df[df['quantity'] > 0  & (df['price_per_unit'] > 0)]
df['calculated_total'] = df['quantity'] * df['price_per_unit']
df = df[df['total_amount'] == df['calculated_total']]
df.drop('calculated_total', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    1000 non-null   int64         
 1   date              1000 non-null   datetime64[ns]
 2   customer_id       1000 non-null   string        
 3   gender            1000 non-null   object        
 4   age               1000 non-null   int64         
 5   product_category  1000 non-null   object        
 6   quantity          1000 non-null   int64         
 7   price_per_unit    1000 non-null   int64         
 8   total_amount      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(2), string(1)
memory usage: 70.4+ KB


In [8]:
print("Nulls: ")
print(df.isnull().sum())
print("\nDuplicates: ")
print(df.duplicated().sum())
print(df['gender'].value_counts())
print(df['customer_id'].nunique())



Nulls: 
transaction_id      0
date                0
customer_id         0
gender              0
age                 0
product_category    0
quantity            0
price_per_unit      0
total_amount        0
dtype: int64

Duplicates: 
0
gender
Female    510
Male      490
Name: count, dtype: int64
1000


In [9]:
df['order_month'] = df['date'].dt.to_period('M')
df['order_month'] = df['order_month'].astype(str)
df['order_year'] = df['date'].dt.year
column_order = ['transaction_id', 'date', 'order_month', 'order_year', 'customer_id', 
                'gender', 'age', 'product_category', 'quantity', 'price_per_unit', 'total_amount']
df = df.reindex(columns=column_order)

df.head()

,transaction_id,date,order_month,order_year,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount
0,1,2023-11-24,2023-11,2023,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,2023-02,2023,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,2023-01,2023,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,2023-05,2023,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,2023-05,2023,CUST005,Male,30,Beauty,2,50,100


In [10]:
df.to_csv('cleaned_retail_sales.csv', index=False)

In [14]:
%pip install sqlalchemy pyodbc


Note: you may need to restart the kernel to use updated packages.


Load the data onto the database

In [12]:
import sqlalchemy as sql 

engine = sql.create_engine("mssql+pyodbc://Jules\\SQLEXPRESS/retail_sales_data?driver=ODBC+Driver+17+for+SQL+Server")
conn = engine.connect()

if conn.closed:
    print("Connection is closed")
else:
    print("Connection is open")



Connection is open


In [13]:
df.to_sql('sales_data', con=conn, if_exists='replace', index=False)

50